In [106]:
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

## Dedicaton Analysis

In [107]:
#input
hrs_per_week = 5 #time_spent_hrs_per_week
weeks_per_year = 20 #tiem_spent_weeks_per_year
period_years_of_activity = 3 #period_years_of_activity

In [108]:
input_values = [float(hrs_per_week), float(weeks_per_year), float(period_years_of_activity)] #입력값이 int라면 float로 변환하기
input_values

[5.0, 20.0, 3.0]

In [109]:
input_values[0]

5.0

In [110]:
input_values[1]

20.0

In [111]:
type(input_values[1])

float

In [112]:
df = pd.read_csv('dedication_analysis .csv')
df

,period_years_of_activity,coefficient,hrs_per_week,week_strength,weeks_per_year,year_strength,total_activity_hrs,total_dedication_avg,mar_fit_score,mjr_fit_8_10,mjr_fit_6_7,mjr_fit_3_4,mjr_fit_1_2,mjr_fit_0
0,5.0,1.00,15.0,5.0,52,5.0,3300.0,5.0,5.0,4.0,3.0,2.0,NaN,NaN
1,4.0,1.00,14.0,4.5,51,5.0,3200.0,4.0,4.0,3.0,2.0,NaN,NaN,NaN
2,3.0,0.90,13.0,4.5,50,5.0,3100.0,4.0,3.0,2.0,NaN,1.0,1.0,NaN
3,2.0,0.75,12.0,4.5,49,5.0,3000.0,4.0,2.0,1.0,1.0,NaN,NaN,NaN
4,1.0,0.60,11.0,4.5,48,5.0,2900.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.10,10.0,4.5,47,5.0,2800.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,9.0,4.0,46,5.0,2700.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,8.0,4.0,45,5.0,2600.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,7.0,3.0,44,5.0,2500.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,6.0,3.0,43,5.0,2400.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN


 dedication_cal = Coefficient X [ (한주별 활동 시간 강도  + 1년간 몇주 참여 강도) / 2 ]


In [113]:
#한주별 활동 시간 강도
for i, row in df.iterrows(): #한행씩 읽어와서
    if df.at[i, 'hrs_per_week'] == input_values[0]: #입력한 값(12.0)과 같은 값이 hrs_per_week 열의 행값과 같다면,
        ws_score = df.at[i, 'week_strength'] # 그 행에서 컬럼이름 week_strength의 해당 값을 가져온다.
    elif df.at[i, 'hrs_per_week'] > input_values[0]:
        ws_score = df.at[0, 'week_strength'] # 값이 없다면, 그 이상되는 값이기 때문에 최대 점수를 줌 5.0
    else:
        pass
ws_score

3.0

In [114]:
#연간 몇 주 참여강도
for k, row in df.iterrows(): #한행씩 읽어와서
    if df.at[k, 'weeks_per_year'] == input_values[1]: #입력한 값(12.0)과 같은 값이 hrs_per_week 열의 행값과 같다면,
        wy_score = df.at[k, 'year_strength'] # 그 행에서 컬럼이름 week_strength의 해당 값을 가져온다.
    elif df.at[k, 'weeks_per_year'] > input_values[1]:
        wy_score = df.at[0, 'year_strength'] # 값이 없다면, 그 이상되는 값이기 때문에 최대 점수를 줌 5.0
    else:
        pass
wy_score

4.5

In [115]:
input_values[2]

3.0

In [116]:
#Coefficient
for l, row in df.iterrows(): #한행씩 읽어와서
    if df.at[l, 'period_years_of_activity'] == input_values[2]: #입력한 값(12.0)과 같은 값이 hrs_per_week 열의 행값과 같다면,
        py_score = df.at[l, 'coefficient'] # 그 행에서 컬럼이름 week_strength의 해당 값을 가져온다.
    elif 6 <= input_values[2]: # 6이면 
        py_score = df.at[0, 'coefficient'] # 값이 없다면, 그 이상되는 값이기 때문에 최대 점수를 줌 5.0
    elif 0 == input_values[2]: # 0이면 
        py_score = 0.1
    else:
        pass
py_score

0.9

## Dedication 계산 (활동별 / 전체적)

In [117]:
# Dedication 계산 (활동별 / 전체적)
dedecation_cal = py_score * ((ws_score + wy_score)/2)
dedecation_cal

3.375

## Leadership 계산 (활동별 / 전체적)  - 1 개의 입력 처리 (def 미적용)

In [118]:
input_act_text = """ notapplicable """

# """cofounder the quality or ability that makes a person a , or the position of being a leader: 
#                     The company was extremely successful under Murphy's leadfffership."""

In [119]:
input_corpus = str(input_act_text) #문장입력
input_corpus = input_corpus.lower()#소문자 변환

sentences  = sent_tokenize(input_corpus) #문장 토큰화
sentences

[' notapplicable']

In [120]:
split_sentences = []
for sentence in sentences:
    processed = re.sub("[^a-zA-Z]"," ", sentence)
    words = processed.split()
    split_sentences.append(words)

In [121]:
comp_txt = sum(split_sentences, [])
comp_txt

['notapplicable']

In [122]:
superb_list = ['founder', 'co-founder', 'cofounder', 'chair', 'president', 'head', 
               'chief', 'first author', 'captain', 'committee head', 'head of board', 
               'board chair', 'chairperson', 'chairman', 'leader', 'CEO','ceo', 'organizer', 
               'director', 'author', 'co-author', 'coauthor', 'chief editor', 'editor in chief']

strong_list = ['founding member', 'VP', 'vp', 'vice Chair', 'vice', 'deputy', 'second author', 'vice captain',
               'committee member', 'board member', 'second author', 'editor']
good_list = ['proctor', 'prefect', 'mediator', 'third author'] # 데이터 추가할 필요가 있음

mediocre_list = ['member', 'participant', 'helper', 'assistant']

weak = ['Not Applicable']




In [123]:
leadership_score = []

for word in comp_txt:
    if word in superb_list:
        print("superb")
        leadership_score.append("5")
        
    elif word in strong_list:
        print("strong")
        leadership_score.append("4")
        
    elif word in strong_list:
        print("good")
        leadership_score.append("3")   
        
    elif word in mediocre_list:
        print("mediocre")
        leadership_score.append("2") 
    elif word == 'notapplicable': # 이게 입력데이터 선택되었을 경우 (웹에서 선택할 수 있도록 구현해야 함! not applicable > notapplicable)
        print("Not Applicable")
        leadership_score.append("Not Applicable") 
    else:
        print("Not Sure")
        leadership_score.append("2") 
        
# 점수확인        
leadership_score

Not Applicable


['Not Applicable']

In [124]:
df_fit_re = pd.DataFrame(leadership_score)

In [125]:
df_fit_re.columns = ['score']

In [126]:
list_fit_re = df_fit_re.drop_duplicates() #중복값 제거!!!! 결과 도출
list_fit_re

,score
0,Not Applicable


In [127]:
type(list_fit_re)

pandas.core.frame.DataFrame

In [128]:
list_fit_re

,score
0,Not Applicable


In [129]:
'2' in list_fit_re.values

False

In [130]:
#조건문을 만들어서 결과를 출력해보자.
fit_anaysis_result_fin =[]
if '5' in list_fit_re.values: # 5이 하나라도 있다면, 5 출력
    print("5")
    fit_anaysis_result_fin.append('5')
elif '4' in list_fit_re.values: # 4 이 있다면 , 4 출력
    print("4")
    fit_anaysis_result_fin.append('4')
elif '3' in list_fit_re.values: # 3 이 있다면 , 3 출력
    print("3")
    fit_anaysis_result_fin.append('3')
elif '2' in list_fit_re.values: # 2 이 있다면 , 2 출력
    print("2")
    fit_anaysis_result_fin.append('2')
elif '1' in list_fit_re.values: # 1 이 있다면 , 1 출력
    print("1")
    fit_anaysis_result_fin.append('1')
elif 'Not Applicable' in list_fit_re.values: # Not Applicable 이 있다면 , Not Applicable 출력
    print("Not Applicable")
    fit_anaysis_result_fin.append('Not Applicable')
else:
    print("NOT SURE")
    fit_anaysis_result_fin.append('NOT SURE')
    
fit_anaysis_result_fin

Not Applicable


['Not Applicable']

## Leadership 계산 (활동별 / 전체적)  - 3 개의 입력 처리 (def 적용)

In [131]:
input_text = """ deputy Member (9th/10th) Treasurer (11th/12th) National Honors Society, Ridgefield High School Chapter
We are amongst the highest academically achieving students at our school, who collectively and consistently participate in community service projects.""" # 실제 값은 문장이 입력되어야 함, 현재는 테스트용 단어입력
input_text_ = """ Leader/Concertmaster (10th-12th)
AMAC Youth Chamber Ensemble (AYCE), audition-based community choir 
Lead ensemble in rehearsal and performance, coordinate rehearsal times, aid younger  """
input_text__ = """ Number 1 Doubles Starter (9th-12th), Captain (11th-12th)
JV/V Beverly Hills High School Tennis Team
Three year League Champions; planned and hosted team banquet; led team warmups and meetings; Coach's Award Recipient (11th); Team Spirit Award (12th).  """

In [132]:
def leadership_analysis(text):
    input_corpus = str(text) #문장입력
    input_corpus = input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(input_corpus) #문장 토큰화
    
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)
        
    comp_txt = sum(split_sentences, [])
    
    superb_list = ['founder', 'co-founder', 'cofounder', 'chair', 'president', 'head', 
               'chief', 'first author', 'captain', 'committee head', 'head of board', 
               'board chair', 'chairperson', 'chairman', 'leader', 'CEO','ceo', 'organizer', 
               'director', 'author', 'co-author', 'coauthor', 'chief editor', 'editor in chief']

    strong_list = ['founding member', 'VP', 'vp', 'vice Chair', 'vice', 'deputy', 'second author', 'vice captain',
                   'committee member', 'board member', 'second author', 'editor']
    good_list = ['proctor', 'prefect', 'mediator', 'third author'] # 데이터 추가할 필요가 있음

    mediocre_list = ['member', 'participant', 'helper', 'assistant']

    weak = ['Not Applicable']
    
    leadership_score = []

    for word in comp_txt:
        if word in superb_list:
            print("superb")
            leadership_score.append("5")

        elif word in strong_list:
            print("strong")
            leadership_score.append("4")

        elif word in strong_list:
            print("good")
            leadership_score.append("3")   

        elif word in mediocre_list:
            print("mediocre")
            leadership_score.append("2") 
        elif word == 'notapplicable': # 이게 입력데이터 선택되었을 경우 (웹에서 선택할 수 있도록 구현해야 함! not applicable > notapplicable)
            print("Not Applicable")
            leadership_score.append("Not Applicable") 
        else:
            print("Not Sure")
            leadership_score.append("2")
            
    df_fit_re = pd.DataFrame(leadership_score)
    df_fit_re.columns = ['score']
    list_fit_re = df_fit_re.drop_duplicates() #중복값 제거!!!! 결과 도출
    
    #조건문을 만들어서 결과를 비교 출력해보자.
    fit_anaysis_result_fin =[]
    if '5' in list_fit_re.values: # 5이 하나라도 있다면, 5 출력
        print("5")
        fit_anaysis_result_fin.append('5')
    elif '4' in list_fit_re.values: # 4 이 있다면 , 4 출력
        print("4")
        fit_anaysis_result_fin.append('4')
    elif '3' in list_fit_re.values: # 3 이 있다면 , 3 출력
        print("3")
        fit_anaysis_result_fin.append('3')
    elif '2' in list_fit_re.values: # 2 이 있다면 , 2 출력
        print("2")
        fit_anaysis_result_fin.append('2')
    elif '1' in list_fit_re.values: # 1 이 있다면 , 1 출력
        print("1")
        fit_anaysis_result_fin.append('1')
    elif 'Not Applicable' in list_fit_re.values: # Not Applicable 이 있다면 , Not Applicable 출력
        print("Not Applicable")
        fit_anaysis_result_fin.append('Not Applicable')
    else:
        print("NOT SURE")
        fit_anaysis_result_fin.append('NOT SURE')

    return fit_anaysis_result_fin
    

In [133]:
# 이 함수를 실행하면 3개의 입력값이 비교되어 계산됨
def start_leadership_analysis(a, b, c):
    input_act_text = a # 실제 값은 문장이 입력되어야 함, 현재는 테스트용 단어입력
    input_act_text_ = b
    input_act_text__ = c
    input_txts = [input_act_text,input_act_text_, input_act_text__]
    
    result = []
    for i in input_txts:
        each_re = leadership_analysis(i)
        result.append(each_re)

    return result

In [134]:
####### 실행 테스트 ####### 잘됨!!!

re_leader = start_leadership_analysis(input_text,input_text_,input_text__) 
result_leadership = sum(re_leader, [])
result_leadership

strong
mediocre
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
4
superb
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
5
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
superb
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
Not Sure
5


['4', '5', '5']

In [135]:
# 가중치 적용 계산

fin_re = round(float(result_leadership[0]) * 0.7
               + float(result_leadership[1]) * 0.2
               + float(result_leadership[2]) * 0.1, 2)
fin_re

4.3

## Description 강도 계산(활동별/전체적)

In [136]:
# 일단 3개의 EXTRACURRICULAR ACTIVITY EXAMPLES  입력, 추가로 활동을 입력할 수 있음. 최대 10개, 그 이상도 가능하지만 비율로 게산

input_text_1 = """ deputy Member (9th/10th) Treasurer (11th/12th) National Honors Society, Ridgefield High School Chapter
We are amongst the highest academically achieving students at our school, who collectively and consistently participate in community service projects.""" # 실제 값은 문장이 입력되어야 함, 현재는 테스트용 단어입력

input_text_2 = """ Leader/Concertmaster (10th-12th)
AMAC Youth Chamber Ensemble (AYCE), audition-based community choir 
Lead ensemble in rehearsal and performance, coordinate rehearsal times, aid younger  """

input_text_3 = """ Number 1 Doubles Starter (9th-12th), Captain (11th-12th)
JV/V Beverly Hills High School Tennis Team
Three year League Champions; planned and hosted team banquet; led team warmups and meetings; Coach's Award Recipient (11th); Team Spirit Award (12th).  """

input_text_4 = """ Peer Advisor (11th-12th)
Erving High School Student Ambassador Program, selective application-based leadership team
Organized and led orientation; served as a year round leader, mentor, tutor, and friend to freshmen; helped with class scheduling."""

input_text_5 = """ Leader (11th)
Books on Global Health Equity and Social Justice, advocacy-focused peer discussion group
Researched global health equity/social justice , assigned weekly readings for group discussion, brainstormed questions to generate input from members.  """

input_text_6 = """ Number 1 Doubles Starter (9th-12th), Captain (11th-12th)
JV/V Beverly Hills High School Tennis Team
Three year League Champions; planned and hosted team banquet; led team warmups and meetings; Coach's Award Recipient (11th); Team Spirit Award (12th).  """

input_text_7 = """ Number 1 Doubles Starter (9th-12th), Captain (11th-12th)
JV/V Beverly Hills High School Tennis Team
Three year League Champions; planned and hosted team banquet; led team warmups and meetings; Coach's Award Recipient (11th); Team Spirit Award (12th).  """

input_text_8 = """Student Coach (9th - 12th)
Middle School MathCounts Team
Taught strategies, selected competitors, hosted weekly practice sessions and lectures. Led team to 2nd place victory at State Mathematics competition (11th). """

input_text_9 = """ Protein Modeling Team Leader (10th)
Science Olympiad, Burke High School Club
Supervised building of protein molecule model, taught peers to use 3D molecular program Jmol; placed in top ten in 2017 regional competition. """

input_text_10 = """""" #이것은 값이 없기 때문에 null로 처리해 보자



In [137]:
# 전체 활동 수 계산(입력한 활동수가 자동으로 카운트 되도록 = 여기서는 기본값으로 3개를 넣어봄)
# 웹에서 숫자가 입력되어야 함(입력한 활동 수)

total_actvity_numbs = len([input_text_1, input_text_2, input_text_3, input_text_4, input_text_5,
                          input_text_6,input_text_7, input_text_9, input_text_10])
total_actvity_numbs # 10번값이 없으니까 9로 나옴, 입력된 값 만큼 계산됨

9

In [138]:
# 캐릭터(문자) 수 계산
def character_counter(text):

    input_corpus = str(text) #문장입력
    input_corpus = input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(input_corpus))# 총 단어수
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)
    
    re__ = sum(split_sentences,[])
    result = len(re__)
        
    return result

In [139]:
character_counter(input_text)

34

In [140]:
# action verbs 수 계산
def action_verbs_counter(text):
    
    #csv 파일에서 단어 사전 불러오기
    data_action_verbs = pd.read_csv('actionverbs.csv')
    data_ac_verbs_list = data_action_verbs.values.tolist()
    verbs_list = [y for x in data_ac_verbs_list for y in x]
    #print(verbs_list)
    
    # 입력문장 처리
    input_corpus = str(text) #문장입력
    input_corpus = input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(input_corpus) #문장 토큰화
    total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
    total_words = len(word_tokenize(input_corpus))# 총 단어수
    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)
    
    re_act_verbs = sum(split_sentences,[])
    
    ext_re = []
    for actv_item in verbs_list:
        for sent_item in re_act_verbs:
            if actv_item  == sent_item:
                ext_re.append(actv_item)
    
    return ext_re

In [141]:
action_verbs_counter(input_text)

['consistently', 'participate', 'service']

## Major Fit analysis

In [142]:
# 본 코드 ai_major_award.py 는 전공 3개와, 수상명과 수상내역을 입력받아 계산해준다.

# 1) 전공적합성을 계산한다. 결과는 FIT, NOT FIT, NOT SURE 중 하나로 출력된다.
# 2) 수상내역을 토대로 취득점수를 계산한다. 점수는 5점이 만점이다.
#    - 수상명칭이 데이터베이스와 일치하지 않을 경우, 문장/단어 속성을 분석해서 결과를 계산하도록 알고리즘화 되어 있다.


# /*결과예시*/

# ===============전공과 수상내역을 비교한 '전공 적합성' 분석 결과 ================
# Check Major FIT : : ['FIT']
# ==============입력 전공 3개, 수상내역과 수상결과 ===================
# Selected 3 Majors :   Asian Studies, Psychology, Political Science/Government
# Award Title & Prestige :   WSDC, Extremely Selective
# ================== 수상 내역 점수 =======================
# SCORE :  [5.0]

In [144]:

import numpy as np
import re
import pandas as pd


def major_analysis(input_text_major):

    #소문자로 변환
    major_list_re = []
    for mjl in input_text_major:
        mjl_ = mjl.lower()
        mjl__ = mjl_.strip()
        major_list_re.append(mjl__)

    #print (major_list_re)
    return major_list_re

def award_anaysis(input_text_award):

    # 아래와 같이 입력해야 함, 소문자 처리, 공백처리 완료!
    #    Chemical Engineering, investment,biomedical  
    #awards = """ winner of ABC competition competition, none """
    # 2개의 입력: 수상내역, 성적   carnegie international student essay contest	    winner of ABC competition competition

    #입력 문자를 소문자로 변환
    honor_list_re = []
    for hor in input_text_award:
        hor_ = hor.lower()
        hor__ = hor_.strip()
        honor_list_re.append(hor__)

    #print (honor_list_re)
    return honor_list_re


def input_values(inp_majors, inp_awards):
    inp_mjr_value = inp_majors.split(',')
    inp_awd_value = inp_awards.split(',')
    #print (inp_mjr_value)
    #print (inp_awd_value )
    mjr_awd_input = inp_mjr_value + inp_awd_value #입력받은 두개의 리스트를 하나로 합친다
    return mjr_awd_input






##########  START !  ##################################################################
## 1. 웹사이트에서 다음과 같은 2개의 텍스트 값을 입력 받는다. 
# 입력받는 값은 major, awards 이다.
########## 입력 ##########
# 3개의 입력: 전공 3개(3 Majors)를 ,로 구분하여 입력 받아야 함
majors = """ Asian Studies, Psychology, Political Science/Government  """ 
# 2개의 입력: 수상명(Award Title) 수상결과(Prestige) 2개를 ,로 구분하여 입력 받아야 함
awards = """ WSDC, Extremely Selective """ 



# 2. 아래 3개의 함수가 실행된다. 
if __name__ == '__main__':
    get_values_re = input_values(majors, awards)
    major_list = major_analysis(get_values_re[:3]) #['asian studies', 'film studies', 'archeology']
    honor_list = award_anaysis(get_values_re[3:]) #['carnegie international student essay contest', 'extremely selective']

#######################################################################################



# 3. 이제 계산이 시작된다. 설명을 하자면, 데이터프레임을 불러와서 입력된 키워드와 비교하고 결과값을 다양한 방법으로 계산해서 결과를 도출한다.

#Awards 데이터 불러오기
data_awards = pd.read_csv('awards_list.csv')
data_awards['award_title'] = data_awards['award_title'].str.lower() #소문자로 변환해야 타이틀을 비교할 수 있음(매칭되는지)
data_awards['Big Major Category #1'] = data_awards['Big Major Category #1'].str.lower()
data_awards['Big Major Category #2'] = data_awards['Big Major Category #2'].str.lower()
data_awards['Big Major Category #3'] = data_awards['Big Major Category #3'].str.lower()
data_awards['Big Major Category #4'] = data_awards['Big Major Category #4'].str.lower()

is_honor_list = data_awards['award_title'] == honor_list[0] # 입력한 결과중 수상내역 문자열 과 데이터프레임의 타이클 컬럼에서 같은 조건
re_honor = data_awards[is_honor_list] # 데이터 필터링하여 새로운 변수에 저장

#print (re_honor) # 결과 출력(여기서 결과가 나오면, 수상내역 출력, 없으면... )

award_anaysis_re = re_honor[["award_title","prestige_score","Big Major Category #1","Big Major Category #2","Big Major Category #3","Big Major Category #4"]]

#print (award_anaysis_re)



# major fit을 계산하려면 입력한 희망전공 3개를 예: ['mechanical engineering', ' computer science', ' bioligy']

# 각 전공을 다음 리스트에서 검색하여 매칭되는 값이 일치(여기서는 business이면) 하면 'FIT',  아니라면 ''NOT FIT 없으면 '?'로 표시


tech_eng_ = ['mechanical engineering', 'general Engineering', 'mechanical Engineering', 'Industrial engineering', 'bioengineering',
            'operations Research', 'materials Science', 'electrical Engineering', 'computer Science/engineering','computer engineering'
            'civil engineering', 'aerospace Engineering', 'chemical Engineering', 'environmental engineering', 'stem', 'steam', 'robotics', 
            'robots', 'computer', 'computers', 'computer programming', 'programming', 'coding', 'computing', 'mechanical', 'mechanics', 
            'mechanism', 'ai', 'artificial intelligence', 'sensor', 'environmental engineering', 'environmental technology', 
            'alternative energy', 'renewable energy', 'material science', 'biomedical', 'biomaterials', 'nanotech', 'pharmaceutical',
            'biomechanics', 'biomimicking', 'biomimicry', 'machine', 'bioengineering', 'chemical engineering', 'applied physics', 
            'operations research', 'civil engineering', 'aerospace', 'system', 'operations research', 'industrial engineering', 'computer Science'
            'invention', 'electrical', 'computational']

tech_eng = []
for i in tech_eng_:
    i_ = i.lower()
    tech_eng.append(i_)

math_sci_ = ['Agriculture', 'Physics', 'Biology', 'Biophysics', 'Biochemistry/Molecular Biology',
            'Mathematics', 'Applied Mathematics', 'Nutrition/Food Science', 'Genetics', 'Astronomy/Astrophysics',
            'Statistics', 'Chemistry', 'Neuroscience', 'Economics','Nursing','Pre-Med', 'Pre-Veterinary'
            'Pharmacy/Pre-Pharmacy', 'Pre-Dental', 'Pharmacy', 'Pre-Pharmacy', 'STEM', 'STEAM', 'Chemistry', 'physics',
            'biology', 'bio', 'biochem', 'biochemistry', 'science', 'math', 'mathematics', 'algorithm', 'biophysics', 
            'applied math', 'genetics', 'scientific', 'astronomy', 'astrophysics', 'statistics', 'neuroscience', 'dental',
            'doctor', 'physical', 'medical', 'scientific research', 'Biomaterials', 'material science', 'nutrition', 'clinical',
            'earth science', 'environmental science', 'ecology', 'ecological', 'veterinary', 'molecular', 'nanotech', 'biomedical',
            'pharmaceutical', 'food science, biotech', 'brain', 'botany', 'botanical', 'immunology', 'immune', 'zoology', 'biomechanics',
            'biomimicking', 'biomimicry']

math_sci = []
for j in math_sci_:
    j_ = j.lower()
    math_sci.append(j_)
    
social_sci_ = ['Economics Asian Studies', 'African Studies','Psychology', "Women's Studies",'Anthropology', 'Political Science/Government',
                'Political Science', 'Political Government', 'International Relations/Affairs', 'International Relations',
                'International Affairs', 'Public Policy/Affairs', 'Public Policy', 'Public Affairs', 'Public Health',
                'Environmental Science/Studies', 'Environmental Science', 'Environmental Studies', 'Urban Planning', 'Education',
                'Volunteer', 'community', 'community service', 'culture', 'equality', 'service',' social issue', 'social', 
                'society', 'economic', 'economy', 'SDGs', 'environment', 'policy', 'psychology', 'Asian studies', 'race relations', 
                'ethnicity', 'ethnic', 'women', 'anthropology', 'politic', 'international relations', 'international affairs',
                'diplomacy', 'diplomat', 'public policy', 'public health,' 'inequality', 'geography', 'geographic', 'linguistic', 
                 'sociology', 'culture', 'urban studies', 'diversity', 'LGBT', 'LGBTQ', 'sustainable', 'sustainability', 'debate',
                 'Middle East',' Middle Eastern Studies']

social_sci = []
for k in social_sci_:
    k_ = k.lower()
    social_sci.append(k_)

humanities_ = ['Archeology', 'Film Studies','German (Language and Literature)','Art History',
                'French (Language and Literature)','Spanish','Theology/Religious Studies',
                'Linguistics','History','English Language and Literature','Comparative Literature',
                'Philosophy','Classical Studies/Latin/Greek', 'English', 'literature', 'history', 'writing', 
              'heritage', 'essay', 'script', 'culture' ,'humanities', 'poem', 'poet', 'playwright', 'script',
              'poetry', 'literary', 'lit', 'writing', 'Spanish', 'German', 'Chinese', 'Italian', 'French', 'Russian', 'literature', 
              'Latin', 'Greek', 'anthropology', 'archaeology', 'Law', 'ethics', 'film studies']

humanities = []
for l in humanities_:
    l_ = l.lower()
    humanities.append(l_)
    

visualart_ = ['Film&Television', 'Film', 'Television', 'Design (Graphic, Industrial, Computer Graphics)','Graphic Design','Industral Design', 
             'Computer Graphics', 'Fashion Design', 'Photography', 'Architecture', 'Movie', 'visual', 'music', 'art', 'arts', 'performing', 
             'paint', 'painting',' visual arts', 'design', 'draw', 'drawing', 'graphic', 'architecture', 'architect', 'film', 'sculpture',
             'artwork', 'fine art']

visualart = []
for m in visualart_:
    m_ = m.lower()
    visualart.append(m_)
    

business_ = ['Business', 'Finance', 'Accounting', 'Marketing', 'Intl', 'Biz','International Business','Mgmt.', 'Info Systems', 
            'Hotel/Hospitality Management', 'Hotel Management', 'Hospitality Management', 'Business', 'start up', 'start-up',
            'entrepreneur', 'entrepreneurship', 'investment', 'invest', 'business plan', 'finance', 'marketing',' organizational behavior',
            'logistics', 'management', 'managing', 'social entrepreneur', 'social enterprise', 'accounting', 'managerial', 'MIS', 
            'Management Information Systems', 'Hotel management', 'hospitality']

business = []
for o in business_:
    o_ = o.lower()
    business.append(o_)
    

music_ = ['Music (Performance)', 'Music', 'Music Theory']

music = []
for p in music_:
    p_ = p.lower()
    music.append(p_)
    

performing_arts_ = ['performing arts','performance', 'dance', 'Movie', 'music', 'musical', 'theater', 'theatre', 'opera', 'thespian', 'song',
                   'sing', 'vocal', 'drama', 'orchestra', 'band', 'acting', 'acrobatics', 'ballet','circus', 'magic', 'mime', 'puppetry',
                   'ventriloquism', 'spoken word', 'stand-up comedy', 'Chinese opera', 'pansori', 'chamber music', 'cabaret', 
                   'jazz band', 'big band', 'Bharatanatyam', 'Gamelan semar pegulingan', 'Shōmyō', 'Odissi', 'Cantonese opera']

performing_arts = []
for q in performing_arts_:
    q_ = q.lower()
    performing_arts.append(q_)

communication_ = ['Documentary', 'SNS', 'Social Network', 'PR', 'public relations', 'advertisement', 'advertising', 'movie', 'film', 'news',
                 'journalism', 'newspaper', 'debate', 'speech', 'newspaper', 'media', 'communication']

communication = []
for r in communication_:
    r_ = r.lower()
    communication.append(r_)


df_a = pd.DataFrame(tech_eng, columns=["major"])
df_a["main_major"] = "tech_eng"

df_b = pd.DataFrame(math_sci, columns=["major"])
df_b["main_major"] = "math_sci"

df_c = pd.DataFrame(social_sci, columns=["major"])
df_c["main_major"] = "social_sci"

df_d = pd.DataFrame(humanities, columns=["major"])
df_d["main_major"] = "humanities"

df_e = pd.DataFrame(business, columns=["major"])
df_e["main_major"] = "business"

df_f = pd.DataFrame(music, columns=["major"])
df_f["main_major"] = "music"

df_g = pd.DataFrame(performing_arts, columns=["major"])
df_g["main_major"] = "performing_arts"

df_h = pd.DataFrame(communication, columns=["major"])
df_h["main_major"] = "communication"

# 전공별 데이터와 주 전공을 하나의 데이터프레임으로 합친다. 
total_major_df = pd.concat([df_a,df_b,df_c,df_d,df_e,df_f,df_g,df_h])


#검색 결과 추출
major_ctgs_ =[]
major_ctg = list(award_anaysis_re["Big Major Category #1"])
major_ctgs_.append(major_ctg)
major_ctg = list(award_anaysis_re["Big Major Category #2"])
major_ctgs_.append(major_ctg)
major_ctg = list(award_anaysis_re["Big Major Category #3"])
major_ctgs_.append(major_ctg)
major_ctg = list(award_anaysis_re["Big Major Category #4"])
major_ctgs_.append(major_ctg)

major_ctgs = [y for x in major_ctgs_ for y in x]

#print ("major category : ", major_ctgs)

#print (major_list)


# 원하는 전공 리스트의 값을 데이터프레임 total_major_df에서 찾아보자
fit_analysis_list = []
for search_major in major_list:
    
    re__ = total_major_df[total_major_df['major'] == search_major]  # 데이터프레임에 찾고자하는 전공이 있다면

    #print (re__) # 결과물을 리스트에 저장해보자
    fit_analysis_list.append(re__)

#fit_analysis_list 를 문자로 만들어서 개별 리스트로 만들자
s =" ".join(map(str, fit_analysis_list))

fit_list_str = s.split()

#print (fit_list_str)

#드디어 분석 fit analysis!!!!
#선택한 전공 3개, 수상내역 1개를 비교하여 데이터베이스 비교 분석하여 상과 전공이 일치하는지 판단!
fit_result = []


#드디어 분석 fit analysis!!!!
#선택한 전공 3개, 수상내역 1개를 비교하여 데이터베이스 비교 분석하여 상과 전공이 일치하는지 판단!
fit_result = []


# 수상내역이 데이터에 없을 경우 not sure. 그러나 수상점수는 계산이 가능함(first, winner 등의 단어를 확인해서)
if  re_honor.isnull().any().any(): #이 re_honor 이 없으면 false 라면 else 실행되고 not sure로 해야 함
    for category in fit_list_str: #수상내역으로 분석 추출한 결과 내역들을 모두 단어로 쪼개어서 하나씩 추출해서 비교
        for major_item in major_ctgs:# 신청한 전공 3개 꺼내와서 하나씩 비교
            if major_item  == category: # 전공이 있으면 fit
                #print ("fit")
                fit_result.append('fit')

            else: # 원하는 전공 3개와 수상내역과 연관된 전공중 아무것도 매칭되는 것이 없으면 not sure
                    #print ("not fit")
                    fit_result.append('not fit')    
else:
    #print ("not sure")
    fit_result.append('not sure')


#print(fit_result) #데이터프레임으로
df_fit_re = pd.DataFrame(fit_result)
df_fit_re.columns = ['Calssfy_fit']
list_fit_re = df_fit_re.drop_duplicates() #중복값 제거!!!! 

#조건문을 만들어서 결과를 출력해보자.
fit_anaysis_result_fin =[]
if 'fit' in list_fit_re.values : # fit이 하나라도 있다면, FIT 출력
    #print("FIT")
    fit_anaysis_result_fin.append('FIT')
elif 'not fit' in list_fit_re.values : # not fit 이 있다면 , NOT FIT 출력
    #print("NOT FIT")
    fit_anaysis_result_fin.append('NOT FIT')
else:
    #print("NOT SURE")
    fit_anaysis_result_fin.append('NOT SURE')
    
fit_anaysis_result_fin

print ("===============전공과 수상내역을 비교한 '전공 적합성' 분석 결과 ================")
print("Check Major FIT : :", fit_anaysis_result_fin)


# ########################### 2.수상내역에 대한 점수를 계산한다.

#수상리스트의 값을 점수로 계산해보자  extremely selective 같은 경우는 그냥 데이프레임에서 추출하면 됨
#단, winner of ABC competition competition, none  입력을 받았을 경우에는 데이터베이스에 수상내역이 없기 때문에 예외처리를 해야 한다.
is_honor_list = data_awards['award_title'] == honor_list[0] # 입력한 결과중 수상내역 문자열 과 데이터프레임의 타이클 컬럼에서 같은 조건
re_honor = data_awards[is_honor_list] # 데이터 필터링하여 새로운 변수에 저장
re_honor # 결과 출력(여기서 결과가 나오면출력, 만약 수상내역 출력, 없으면... 예외처리를 해야함.. 2번째 방식으로 분석해야 함)
#점수를 추출한다. 그리고 이 점수는 입력한 수상내역의 문자를 확인하여 '예외의 룰'을 적용!!!
award_anaysis_re = re_honor[["prestige_score"]] #이건 데이터프레임
#print(award_anaysis_re) # 5.0 이 나옴
fit_re_fin = sum(award_anaysis_re.values.tolist(),[]) #리스트로 변환
# print ("==== 수상내역 점수 ==================================")
# print ("수상점수_1 : ", fit_re_fin)  #결과값 도출



award_anaysis_re_TF = award_anaysis_re.isnull().values.any()#데이터 프레임에 값의 유무 확인. 부울 값을 반환한다. 값이 있으면  Fasle???이다.

#print(award_anaysis_re_TF) # False가 나옴

print ("==============입력 전공 3개, 수상내역과 수상결과 ===================")
#입력값 확인
print ("Selected 3 Majors : ", majors)
print ("Award Title & Prestige : ", awards)

#input awards : winner of ABC competition competition, none   >>>>>>>>>>> award_anaysis_re_TF == True 로 할 경우 수상점수_2 가 실행됨 <성공>
#input awards : winner of ABC competition competition, none   >>>>>>>>>>> award_anaysis_re_TF == False 로 할 경우 수상점수_1 이 실행됨

#input awards : carnegie international student essay contest, Extremely Selective >>>>> award_anaysis_re_TF == True 로 할 경우 수상점수_2 가 실행됨 

if not fit_re_fin:  ########## 여기 조건문을 고쳐야 함...
    #### 3.수상내역이 DB에 없을 경우에 점수 계산하기
    print ("================== 수상 점수 계산 ===================")
    print ("수상내역에 DB에 없습니다. 그래서 단어 분석을 통해서 점수를 계산해 볼께요.")
    remove_null_text = awards.strip() #앞 뒤 공백 제거한다. 입력할 때 공백을 넣을 수 있기 때문에.. 에러방지
    split_awards_list = remove_null_text.split(" ")
    split_awards_list #결과를 토대로 점수 계산하기위해서 data_awards_score의 데이터 프레임에서 해당 점수값 도출하기

    data_awards_score = pd.read_csv('award_title_score.csv')
    data_awards_score ['title'] = data_awards_score ['title'].str.lower() #소문자로 변환해야 타이틀을 비교할 수 있음(매칭되는지)
    data_awards_score.set_index('title', inplace=True) #title을 인뎃스로 변환, 그래야 값을 찾기 쉽다.

    #입력한 수생내역의 리스트 값을 하나씩 불러와서 데이터프레이에 있는지 비교 찾아내서 해당 점수를 가져오기
    get_score__ = []
    for award_word in split_awards_list: #데이터프레임에서 인덱스의 값과 비교하여
        if award_word in data_awards_score.index: #df에 특정 단어가 있다면, 해당하는 컬럼의 값을 가져오기
            get_score_ = data_awards_score.loc[award_word,'fin_score']
            get_score__.append(get_score_)
            #print ("got it")
            
        else:
            #print ("The result cannot be calculated. Please enter more information about your award.")
            pass
    print ("================== 수상 점수 계산 =======================")
    print ("SCORE : ", get_score__)

else:
    # 이것이 데이터베이스에 있는 수상명에 해당하는 점수 계산 결과다.
    # fit_re_fin

    fit_re_fin = sum(award_anaysis_re.values.tolist(),[])
    print ("================== 수상 내역 점수 =======================")
    print ("SCORE : ", fit_re_fin) 


===============전공과 수상내역을 비교한 '전공 적합성' 분석 결과 ================
Check Major FIT : : ['FIT']
==============입력 전공 3개, 수상내역과 수상결과 ===================
Selected 3 Majors :   Asian Studies, Psychology, Political Science/Government  
Award Title & Prestige :   WSDC, Extremely Selective 
================== 수상 내역 점수 =======================
SCORE :  [5.0]
